### Spotify Login
After running this click Sign-In and login to Spotify. If the Time expires then the token should refresh using the popup. 

In [ ]:
from ipyauth import ParamsSpotify, Auth

auth = Auth(ParamsSpotify(redirect_uri='http://localhost:8888/callback', client_id="9e4657eefbac41afa98c61f590d8fd51"))
auth

### Common Stuff and Imports


In [ ]:
import requests
from IPython.display import Image
from pandas.io.json import json_normalize
from pandas import DataFrame,read_pickle,merge
from pandas import DataFrame as df
from datetime import datetime,timedelta
import json
from spotify import Spotify

import logging
import os

#user = fetch("/v1/me")
#userId = user["id"]
spot = Spotify(auth)
Image(url=spot.imageUrl, width=100)


### Read The User Library
This process may take a little while. The library tracks are cached locally, so this step can be skipped.

In [ ]:
data = spot.fetchAll("/v1/me/tracks")
tracksDf = json_normalize(data, sep="_").set_index("track_uri")
tracksDf.to_pickle("mytracks.pkl")
tracksDf.head(2)

### Verify the library cache exists

In [ ]:
tracksDf = read_pickle("mytracks.pkl")
tracksCache = tracksDf[["added_at","track_name","track_album_name","track_album_id"]]
tracksCache.head(2)

### Read the Artists
Load the artists one by one. Use the Pickle File **artists.pkl** as a cache.
* Create a DataFrame using the cache file if one exists (otherwise None)
* Get the list of all unique artist_ids from the previous DF
* Call `fetchAllIds` - using the artists path, the `artists` JSON field path and the cache DF
* Recreate the new artistDf (Dict returned)
* Save the file back to **artists.pkl**

### Create a Track to Artist Table
Pick out the track.artists array for each library track record. The meta (parent record) is the track.id. Use a prefix for both the meta and the record because they both use id.


In [ ]:
artist_and_track = json_normalize( data=data, record_path=['track','artists'],  meta=[["track","name"],["track","uri"]],  record_prefix='artist_',   sep="_" )
artist_and_track = artist_and_track[['track_name','artist_id','artist_name', 'track_uri']]

artistsPickle = read_pickle("artists.pkl") if (os.path.isfile("artists.pkl")) else None 
artistIds = list(set(artist_and_track["artist_id"].values))
artists = spot.fetchAllIds("/v1/artists","artists",artistIds,existingDf=artistsPickle)
artistsDf = json_normalize(artists).set_index("id")
artistsDf.to_pickle("artists.pkl")

artistsDf[["name","genres"]].head(2)


### Read the Albums

In [ ]:
albumsPickle = read_pickle("albums.pkl") if (os.path.isfile("albums.pkl")) else None 
album_ids  = list(set(tracksDf["track_album_id"].values))
albums = spot.fetchAllIds("/v1/albums","albums",album_ids,pageSize=20,existingDf=albumsPickle)
albumsDf = json_normalize(albums, sep="_").set_index("id")
albumsDf.to_pickle("albums.pkl")

albumsDf["released"] = albumsDf.apply(lambda al: datetime.strptime(al["release_date"], "%Y" if (al.release_date_precision=="year") else "%Y-%m" if (al.release_date_precision=="month") else "%Y-%m-%d"), axis=1) 

libraryWithAlbums = merge(tracksDf,albumsDf, left_on="track_album_id", right_index=True, suffixes=("_track","_album"))
libraryWithAlbums[["name","release_date","tracks.total","released"]].head(2)


### Track Features in Library

In [ ]:
featuresPickle = read_pickle("features.pkl").set_index("id") if (os.path.isfile("features.pkl")) else None 
features = spot.fetchAllIds("/v1/audio-features","audio_features",tracksDf["track_id"].values,pageSize=50,existingDf=featuresPickle)
featuresDf = json_normalize(features, sep="_")
featuresDf.to_pickle("features.pkl")
lib = merge(libraryWithAlbums,featuresDf.set_index("uri"), left_index=True, right_index=True, how="outer")
lib[libraryWithFeatures.track_name=="Spooky - Out of Order Mix"][["track_name","tempo","danceability","loudness","energy","released","valence"]].head(2)

---------------------------------

## All Read - let's create an auto playlist



### Running Playlist 
* Tempo between 160 and 200 (for cadence)
* Energy above 0.6
* Danceability above 0.7

In [ ]:
newPlaylist = lib[ 
#        (libraryWithFeatures.tempo>155) 
#                                  & (libraryWithFeatures.tempo<170) 
                                  (lib.released>(datetime.now()+timedelta(days=-36500))) 
                                  & (lib.energy>0.96 )]
#                                 & (libraryWithFeatures.danceability>0.8 )]
spot.updatePlaylist("Auto Run Fast","Tempo>150 < 190 energy>50",newPlaylist.index.values.tolist())
newPlaylist[["track_name","released","tempo","track_artists","energy","loudness","tempo","danceability"]]


In [ ]:
newPlaylist = lib[lib.released<datetime.strptime("1970","%Y")]
spot.updatePlaylist("auto 1960s","Sounds of the 60s",newPlaylist.index.values.tolist())

newPlaylist = lib[(lib.released>(datetime.strptime("1970","%Y")))
                                  & (lib.released<(datetime.strptime("1980","%Y")))]
spot.updatePlaylist("auto 1970s","Sounds of the 70s",newPlaylist.index.values.tolist())

newPlaylist = lib[(lib.released>(datetime.strptime("1980","%Y")))
                                  & (lib.released<(datetime.strptime("1990","%Y")))]
spot.updatePlaylist("auto 1980s","Sounds of the 80s",newPlaylist.index.values.tolist())

newPlaylist = lib[(lib.released>(datetime.strptime("1990","%Y")))
                                  & (lib.released<(datetime.strptime("2000","%Y")))]
spot.updatePlaylist("auto 1990s","Sounds of the 90s",newPlaylist.index.values.tolist())

newPlaylist = lib[(lib.released>(datetime.strptime("2000","%Y")))
                                  & (lib.released<(datetime.strptime("2010","%Y")))]
spot.updatePlaylist("auto 2000s","Sounds of the 2000s",newPlaylist.index.values.tolist())

newPlaylist = lib[(lib.released>(datetime.strptime("2010","%Y")))
                                  & (lib.released<(datetime.strptime("2020","%Y")))]
spot.updatePlaylist("auto 2010s","Sounds of the 2010s",newPlaylist.index.values.tolist())

newPlaylist = lib[(lib.released>(datetime.strptime("2019","%Y")))               ]
spot.updatePlaylist("2019 So Far","Music from this year",newPlaylist.index.values.tolist())

newPlaylist[["track_name","released","tempo","track_artists","energy","loudness","tempo","danceability"]]
